# LMC-Apriori Algorithm

## Penjelasan

Algoritma Apriori adalah teknik dalam data mining yang digunakan untuk menemukan pola hubungan atau association rules di antara item dalam dataset yang besar. Algoritma ini paling sering digunakan untuk analisis keranjang belanja atau market basket analysis di bidang retail. Algoritma ini memiliki beberapa kekurangan, seperti kompleksitas dalam pemfilteran set kandidat dan proses perbandingan yang berulang. LMC-Apriori memanfaatkan pendekatan "packet pruning" untuk mengurangi jumlah pencarian yang diperlukan dalam lapisan data, sehingga meningkatkan efisiensi secara signifikan dalam situasi dengan variasi besar antar kelompok data.


Aspek utama dari algoritma LMC-Apriori :

- Pengelompokan Paket: Memisahkan kumpulan data berdasarkan elemen minimum yang sama, kemudian mengurutkan dan memfilter paket berdasarkan kriteria tertentu.
- Penghilangan (Pruning): Mengurangi kompleksitas pencarian dengan mengeliminasi kandidat yang tidak memenuhi syarat frekuensi.
- Efisiensi dalam Traversal Data: Memastikan algoritma lebih cepat pada data skala besar dengan traversal yang lebih sedikit.

## Kode

### Algoritma Kode

1. Inisialisasi Data dan Parameter
- Pertama, inisialisasi data transaksi dari file input, lalu tetapkan nilai minimum support (minSup) dan confidence (minConf).
- Data transaksi disiapkan sebagai struktur data yang mudah diakses (dictionary transList) dan menghitung frekuensi awal dari setiap item individu di dataset (freqList).

2. Pencarian Frequent Itemset (genAssociations)
- Langkah Pertama (First Pass): Temukan item tunggal (1-itemset) yang memenuhi ambang batas support.

  - Untuk setiap item tunggal, hitung nilai support (yaitu proporsi transaksi yang mengandung item tersebut).
  - Jika nilai support memenuhi ambang batas minSup, item tersebut dianggap sebagai frequent itemset 1-itemset.

- Iterasi dan Generasi Kandidat Itemset:
  - Algoritme bekerja secara iteratif, mulai dari itemset 1-itemset, lalu mencari 2-itemset, 3-itemset, dan seterusnya.
  - Pada setiap iterasi ke-k, algoritme menggabungkan itemset dari iterasi sebelumnya (k-1-itemset) untuk membentuk kandidat k-itemset baru.
  - Algoritme kemudian menghitung support untuk setiap kandidat k-itemset.
  - Jika kandidat k-itemset memenuhi support minimum, kandidat tersebut dianggap sebagai frequent itemset dan disimpan.
  - Proses ini berulang hingga tidak ada lagi kandidat itemset yang memenuhi ambang batas.

- Pruning dan Skyline Reduction:
  - Pruning: Algoritme menghapus itemset yang tidak memenuhi ambang batas support.
  - Skyline Reduction: Menghilangkan subset dari itemset yang lebih besar agar hanya menyimpan itemset maksimal yang sering. Hal ini menghindari penyimpanan itemset yang kurang relevan (subset) yang telah diwakili oleh itemset yang lebih besar.

3. Generasi Kandidat Itemset (candidateGen)
- Kandidat itemset baru dibentuk dengan menggabungkan itemset yang sering dari iterasi sebelumnya.
- Setiap kandidat diperiksa apakah memiliki subset yang juga merupakan itemset yang sering. Kandidat yang tidak memenuhi syarat ini dibuang.
- Algoritme ini memastikan hanya kandidat itemset yang relevan yang dihitung frekuensinya dalam dataset.

4. Menghasilkan Association Rules (genRules)
- Setelah menemukan semua frequent itemsets, algoritme menghasilkan association rules.
- Untuk setiap frequent itemset yang terdiri dari 2 item atau lebih, algoritme membuat aturan asosiasi dengan membagi itemset menjadi antecedent (sebelum aturan) dan consequent (setelah aturan).
- Menghitung Confidence: Algoritme menghitung confidence untuk setiap aturan.
  - Confidence adalah rasio antara support dari keseluruhan itemset dengan support dari subset (antecedent).
- Jika confidence memenuhi ambang batas minConf, aturan disimpan sebagai aturan asosiasi yang signifikan.

5. Output dan Tampilan Hasil
- Setelah semua frequent itemsets dan aturan asosiasi dihasilkan, hasilnya dicetak.
- Fungsi readable() digunakan untuk menampilkan itemset dan aturan asosiasi dalam format yang lebih mudah dibaca.

Di bawah ini adalah contoh kode algoritma LMC-Apriori pada studi kasus transaksi toko roti yang diambil dari https://github.com/timothyasp/apriori-python.git dengan beberapa penyesuaian agar dapat dijalankan menggunakan python 3.0 dan Google Collab. Data yang digunakan adalah data transaksi sebanyak 1000, dengan item berjumlah 49 macam. Jenis item dapat dilihat dalam file goods.csv. Dalam kode di bawah ini, penentuan *association rules* didasarkan pada id item dalam dataset 1000-out1.csv. Adapun kode di bawah juga dapat diimplementasikan kepada dataset yang berukuran lebih besar, seperti 2000 data transaksi (2000-out1.csv), 5000 data transaksi (5000-out1.csv), maupun 10000 data transaksi (10000-out1.csv).

Untuk menjalankan kode di bawah, dibutuhkan *file* tambahan yaitu dataset transaksi dan data item yang berupa csv. Kedua *file* ini dapat diakses di https://github.com/timothyasp/apriori-python.git.

In [9]:
# Menunjukkan dataset transaksi
!head -n 10 data/1000/1000-out1.csv # dapat diubah sesuai dengan nama file

1, 7, 15, 44, 49
2, 1, 19
3, 1, 19
4, 3, 4, 15, 18, 35, 44
5, 2, 4, 7, 9, 23
6, 14, 21, 44
7, 4, 12, 31, 36, 44, 48
8, 15, 27, 28
9, 2, 28
10, 3, 18, 35


In [7]:
# Menunjukkan dataset item
!head -n 10 goods.csv # dapat diubah sesuai dengan nama file

0,'Chocolate','Cake',8.95,'Food'
1,'Lemon','Cake',8.95,'Food'
2,'Casino','Cake',15.95,'Food'
3,'Opera','Cake',15.95,'Food'
4,'Strawberry','Cake',11.95,'Food'
5,'Truffle','Cake',15.95,'Food'
6,'Chocolate','Eclair',3.25,'Food'
7,'Coffee','Eclair',3.5,'Food'
8,'Vanilla','Eclair',3.25,'Food'
9,'Napoleon','Cake',13.49,'Food'


In [10]:
import sys
import os.path
import csv
import math
import types
from collections import defaultdict
from collections.abc import Iterable
import itertools

# Kelas Apriori untuk menemukan frequent itemset dan menghasilkan aturan asosiasi
class Apriori:
    def __init__(self, data, minSup, minConf):
        # Inisialisasi variabel utama
        self.dataset = data
        self.transList = defaultdict(list)  # Menyimpan daftar transaksi
        self.freqList = defaultdict(int)  # Menyimpan frekuensi item
        self.itemset = set()  # Menyimpan item yang unik
        self.highSupportList = list()  # Menyimpan item dengan support tinggi
        self.numItems = 0  # Total jumlah item
        self.prepData()  # Menyiapkan data transaksi

        self.F = defaultdict(list)  # Menyimpan frequent itemset

        self.minSup = minSup  # Nilai minimum support
        self.minConf = minConf  # Nilai minimum confidence

    # Menghasilkan asosiasi untuk frequent itemset
    def genAssociations(self):
        candidate = {}
        count = {}

        # Pemindaian pertama untuk menemukan item satu-item yang sering
        self.F[1] = self.firstPass(self.freqList, 1)
        k = 2
        while len(self.F[k-1]) != 0:  # Iterasi hingga tidak ada lagi frequent itemset
            candidate[k] = self.candidateGen(self.F[k-1], k)  # Menghasilkan kandidat itemset
            # Menghitung frekuensi setiap kandidat pada data transaksi
            for t in self.transList.items():
                for c in candidate[k]:
                    if set(c).issubset(t[1]):
                        self.freqList[c] += 1

            self.F[k] = self.prune(candidate[k], k)  # Menghapus itemset yang tidak memenuhi syarat
            if k > 2:
                self.removeSkyline(k, k-1)  # Menghilangkan subset yang tidak perlu
            k += 1

        return self.F

    # Menghapus subset yang tidak perlu dari frequent itemset
    def removeSkyline(self, k, kPrev):
        for item in self.F[k]:
            subsets = self.genSubsets(item)
            for subset in subsets:
                if subset in self.F[kPrev]:
                    self.F[kPrev].remove(subset)

    # Memfilter itemset berdasarkan nilai support
    def prune(self, items, k):
        f = []
        for item in items:
            count = self.freqList[item]
            support = self.support(count)
            # Jika support >= 0.95, tambahkan ke daftar highSupportList
            if support >= .95:
                self.highSupportList.append(item)
            # Jika memenuhi minimum support, tambahkan ke frequent itemset
            elif support >= self.minSup:
                f.append(item)

        return f

    # Menghasilkan kandidat itemset dengan menggabungkan itemset sebelumnya
    def candidateGen(self, items, k):
        candidate = []

        if k == 2:
            # Menggabungkan dua item jika panjangnya dua
            candidate = [tuple(sorted([x, y])) for x in items for y in items if len((x, y)) == k and x != y]
        else:
            # Menggabungkan itemset yang lebih besar
            candidate = [tuple(set(x).union(y)) for x in items for y in items if len(set(x).union(y)) == k and x != y]

        # Memfilter kandidat yang tidak memenuhi syarat
        for c in candidate:
            subsets = self.genSubsets(c)
            if any([x not in items for x in subsets]):
                candidate.remove(c)

        return set(candidate)

    # Menghasilkan semua subset dari suatu itemset
    def genSubsets(self, item):
        subsets = []
        for i in range(1, len(item)):
            subsets.extend(itertools.combinations(item, i))
        return subsets

    # Menghasilkan aturan asosiasi dari frequent itemset
    def genRules(self, F):
        H = []

        for k, itemset in F.items():
            if k >= 2:
                for item in itemset:
                    subsets = self.genSubsets(item)
                    for subset in subsets:
                        # Mendapatkan count subset dan item
                        if len(subset) == 1:
                            subCount = self.freqList[subset[0]]
                        else:
                            subCount = self.freqList[subset]
                        itemCount = self.freqList[item]
                        # Menghitung confidence dan menambahkan aturan jika memenuhi syarat
                        if subCount != 0:
                            confidence = self.confidence(subCount, itemCount)
                            if confidence >= self.minConf:
                                support = self.support(self.freqList[item])
                                rhs = self.difference(item, subset)
                                if len(rhs) == 1:
                                    H.append((subset, rhs, support, confidence))

        return H

    # Menghitung perbedaan antara item dan subset
    def difference(self, item, subset):
        return tuple(x for x in item if x not in subset)

    # Menghitung nilai confidence dari suatu aturan
    def confidence(self, subCount, itemCount):
        return float(itemCount) / subCount

    # Menghitung nilai support dari suatu itemset
    def support(self, count):
        return float(count) / self.numItems

    # Pemindaian pertama untuk item satu-item yang sering
    def firstPass(self, items, k):
        f = []
        for item, count in items.items():
            support = self.support(count)
            if support == 1:
                self.highSupportList.append(item)
            elif support >= self.minSup:
                f.append(item)

        return f

    # Menyiapkan data transaksi menjadi format dictionary
    def prepData(self):
        key = 0
        for basket in self.dataset:
            self.numItems += 1
            key = basket[0]
            for i, item in enumerate(basket):
                if i != 0:
                    self.transList[key].append(item.strip())
                    self.itemset.add(item.strip())
                    self.freqList[(item.strip())] += 1

# Fungsi utama untuk mengelola input dan memulai algoritma
def main():
    goods = defaultdict(list)
    num_args = len(sys.argv)
    minSup = minConf = 0
    noRules = False

    # Memeriksa jumlah argumen input yang benar
    if num_args < 4 or num_args > 5:
        print('Expected input format: python apriori.py <dataset.csv> <minSup> <minConf>')
        return
    elif num_args == 5 and sys.argv[1] == "--no-rules":
        dataset = csv.reader(open(sys.argv[2], "r"))
        goodsData = csv.reader(open('goods.csv', "r"))
        minSup = float(sys.argv[3])
        minConf = float(sys.argv[4])
        noRules = True
        print("Dataset: ", sys.argv[2], " MinSup: ", minSup, " MinConf: ", minConf)
    else:
        dataset = csv.reader(open(sys.argv[1], "r"))
        goodsData = csv.reader(open('goods.csv', "r"))
        minSup = float(sys.argv[2])
        minConf = float(sys.argv[3])
        print("Dataset: ", sys.argv[1], " MinSup: ", minSup, " MinConf: ", minConf)

    print("==================================================================")

    # Membaca deskripsi item dari goods.csv
    for item in goodsData:
        goods[item[0]] = item[1:]

    # Membuat objek Apriori dan menemukan frequent itemset
    a = Apriori(dataset, minSup, minConf)
    frequentItemsets = a.genAssociations()

    # Menampilkan frequent itemset
    count = 0
    for k, item in frequentItemsets.items():
        for i in item:
            if k >= 2:
                count += 1
                print(count, ":  ", readable(i, goods), "\tsupport=", a.support(a.freqList[i]))

    print("Skyline Itemsets: ", count)
    # Jika aturan diminta, menghasilkan dan menampilkan aturan asosiasi
    if not noRules:
        rules = a.genRules(frequentItemsets)
        for i, rule in enumerate(rules):
            print("Rule", i + 1, ":\t ", readable(rule[0], goods), "\t-->", readable(rule[1], goods), "\t [sup=", rule[2], " conf=", rule[3], "]")

    print("\n")

# Fungsi untuk menghasilkan format yang lebih mudah dibaca untuk itemset
def readable(item, goods):
    itemStr = ''
    for k, i in enumerate(item):
        itemStr += goods[i][0] + " " + goods[i][1] + " (" + i + ")"
        if len(item) != 0 and k != len(item) - 1:
            itemStr += ",\t"

    return itemStr.replace("'", "")
'''
# Memulai program jika dijalankan sebagai skrip utama
if __name__ == '__main__':
    main()
'''

# Set parameter
data_path = 'data/1000/1000-out1.csv' # dapat diubah sesuai dengan file dataset
min_sup = 0.03 # dapat diubah sesuai dengan kebutuhan
min_conf = 0.7 # dapat diubah sesuai dengan kebutuhan

# Call main() with simulated command-line arguments
sys.argv = ["apriori.py", '--no-rules', data_path, min_sup, min_conf]
main()

Dataset:  data/1000/1000-out1.csv  MinSup:  0.03  MinConf:  0.7
1 :   Gongolais Cookie (22),	Truffle Cake (5) 	support= 0.058
2 :   Marzipan Cookie (27),	Tuile Cookie (28) 	support= 0.053
3 :   Lemon Cookie (24),	Raspberry Lemonade (41) 	support= 0.03
4 :   Apricot Croissant (32),	Hot Coffee (45) 	support= 0.032
5 :   Casino Cake (2),	Chocolate Coffee (46) 	support= 0.039
6 :   Cheese Croissant (33),	Orange Juice (42) 	support= 0.038
7 :   Lemon Cookie (24),	Lemon Lemonade (40) 	support= 0.031
8 :   Almond Twist (37),	Coffee Eclair (7) 	support= 0.03
9 :   Chocolate Cake (0),	Chocolate Coffee (46) 	support= 0.047
10 :   Strawberry Cake (4),	Napoleon Cake (9) 	support= 0.049
11 :   Raspberry Cookie (23),	Lemon Lemonade (40) 	support= 0.031
12 :   Blueberry Tart (16),	Hot Coffee (45) 	support= 0.033
13 :   Cherry Tart (18),	Opera Cake (3) 	support= 0.041
14 :   Hot Coffee (45),	Apricot Croissant (32),	Blueberry Tart (16) 	support= 0.032
15 :   Chocolate Coffee (46),	Chocolate Cake (0),	C